In [2]:
import pandas_gbq as pdgbq
import pandas as pd
import plotly.express as px

In [3]:
zone_query = '''

SELECT
  pr.pitch_name,
  ROUND(AVG(pv.plate_x), 4) AS avg_X,
  ROUND(AVG(pv.plate_z), 4) AS avg_Z,
  ROUND(AVG(pv.release_speed), 4) AS avg_speed
FROM
  `valuesheet.MLB.pitching_view` AS pv
LEFT JOIN
  `valuesheet.MLB.hitting_data` AS hd
ON
  pv.pitch_id = hd.pitch_id
LEFT JOIN
  `valuesheet.MLB.pitch_ref` AS pr
ON
  pr.pitch_type = pv.pitch_type
WHERE
  hd.events = 'strikeout'
  AND pv.type IN('S')
  AND pv.pitch_type IS NOT NULL
GROUP BY
  pr.pitch_name
ORDER BY
  avg_speed DESC
'''

In [4]:
pitches = pdgbq.read_gbq(query=zone_query,project_id='valuesheet')

Downloading: 100%|██████████| 14/14 [00:00<00:00, 29.95rows/s]


In [5]:
pitches.head(15)

,pitch_name,avg_X,avg_Z,avg_speed
0,4-Seam Fastball,0.0049,2.8371,94.1791
1,2-Seam Fastball,-0.0411,2.4581,93.1223
2,Sinker,0.0012,2.4260,92.9332
3,Cutter,0.1561,2.2120,88.7890
4,Forkball,-0.3047,1.3572,86.5966
5,Split-Finger,-0.1529,1.4439,85.5600
6,Slider,0.2762,1.6779,84.9984
7,Changeup,-0.0685,1.6530,84.5653
8,Knuckle Curve,0.2171,1.4091,82.0891
9,Curveball,0.1296,1.5130,79.3747


In [10]:
fig = px.scatter(pitches, x="avg_X", y="avg_Z", color="pitch_name",
                 title="Hot Zones",
                 labels={"avg_X":"Inside/Outside","avg_Z":"High/Low"},
                 height=500,
                 width=500)
                     
fig.update_layout(legend_title_text='Pitch')

In [11]:
fig.write_html('index.html')